In [1]:
from datasets import load_dataset
def get_acts(model, prompts):
    # The number of layers our model has. GPT2-medium has 24
    layers = range(model.cfg.n_layers)

    # This is going to hold all of our activations. Notice the shape here: [n_prompts, n_layers, d_model]
    data = torch.zeros((len(prompts), len(layers), model.cfg.d_model))

    # For every prompt
    for i, prompt in tqdm(enumerate(prompts)):
        # Do a forward pass with the LLM on said prompt. This function lets us
        # cache the activations.
        _, activations = model.run_with_cache(prompt)

        # For every layer, go through and grab the activation we want at that layer
        # The "[0, -1]" there is just getting the first batch (we do one batch at a time, this
        # could probably be improved) and then the last token at that batch (the last token
        # in the residual stream probably (if some literature is correct) contains the "most
        # information". This is the last token /in the residual stream/, not like "dog" in
        # "John has a dog". We could experiment if this is the right place/token to try but
        # that's for another day
        for j in layers:
            # Store that activation!
            data[i, j] = activations[f'blocks.{j}.hook_resid_post'][0,-1]

    return data

In [ ]:
ds = load_dataset("community-datasets/generics_kb", "generics_kb_best")

In [ ]:
import random
sentences = ds['train']['generic_sentence']

In [ ]:
first_10k = sentences[0:10_000]

acts = get_acts(gpt2_medium, first_10k)

torch.save(acts, "generics_kb_10k_10272024.npy")